Todo:
 1. remove noisy samples: one option - remove rows with num of nan > trash
 2. feature selection (annova, pca) 
 3. check for cat boost 
 4. check SMOTETomek and undersample & SMOTENC
 
Must:
 1. XGBOOST with missing
 2. random oversample 
 3. Net
 4. complete missing with classifier

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from utils import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
##### load & clean data
train, test = load_data()
train = train.dropna(axis=0, how='all') # remove missing rows
y_train = train.Class
train = train.drop(columns='Class')

columns_to_drop = ['Pre_L_Pupil_Day', 'T_L_Actual_AblDepth']
train, test = remove_columns(train, test, columns_to_drop)

print("train.shape without missing rows =", train.shape)
print("class count:\n", y_train.value_counts())

train.shape without missing rows = (30451, 49)
class count:
 0.0    29949
1.0      502
Name: Class, dtype: int64


In [3]:
##### add here new steps to preprocess
preprocessing = Pipeline([
                ('scaler', StandardScaler()),
                ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'))
                ])
X_train, X_test = preprocess_data(train, test, preprocessing)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, stratify=y_train)


In [4]:
##### add here new steps before predict (TODO: undersample)
evaluation =  Pipeline([
                ('smote', SMOTE(random_state=42, ratio=1, n_jobs=-1, k_neighbors=3)),
                ('rf', RandomForestClassifier(n_estimators=50, n_jobs=-1, min_samples_split=2, max_depth=5))
                ])
evaluation.get_params().keys()

dict_keys(['memory', 'steps', 'smote', 'rf', 'smote__k_neighbors', 'smote__kind', 'smote__m_neighbors', 'smote__n_jobs', 'smote__out_step', 'smote__random_state', 'smote__ratio', 'smote__sampling_strategy', 'smote__svm_estimator', 'rf__bootstrap', 'rf__class_weight', 'rf__criterion', 'rf__max_depth', 'rf__max_features', 'rf__max_leaf_nodes', 'rf__min_impurity_decrease', 'rf__min_impurity_split', 'rf__min_samples_leaf', 'rf__min_samples_split', 'rf__min_weight_fraction_leaf', 'rf__n_estimators', 'rf__n_jobs', 'rf__oob_score', 'rf__random_state', 'rf__verbose', 'rf__warm_start'])

In [5]:
#### Cross Validation

n_splits = 10 # number of folds (stratified KFold)

### add here new hyperparameters
rf_hyperparams = {
    'rf__n_estimators': np.arange(start=100, stop=400, step=20),
    'rf__max_depth': list(np.arange(5,30,5)) + [None],
    'rf__min_samples_split': [2, 5, 10]
}
smote_hyperparams = {
    # 'smote__k_neighbors': np.arange(2,10)
    # 'smote__ratio': ??? TODO
}

gs = GridSearchCV(evaluation, 
                  param_grid={**rf_hyperparams, **smote_hyperparams},
                  scoring='roc_auc',
                  cv=StratifiedKFold(n_splits=n_splits),
                  n_jobs=-1)

gs.fit(X_train, y_train)  

In [6]:
gs.fit(X_train, y_train)  
# print('best params:\n\t{}'.format(gs.best_params_))
# print('best auc score:\n\t{:.4f}'.format(gs.best_score_))

x_val_pred = gs.predict_proba(X_val)[:,1]
print('auc score on x_val:\n\t{:.4f}'.format(roc_auc_score(y_val, x_val_pred)))
pred = gs.predict_proba(X_test)[:,1]


auc score on x_val:
	0.5489


In [7]:
df = ensemble_with_rokah(pred)

In [8]:
to_file(df, "lol")